In [1]:
%matplotlib inline
# %load ../../utility/initialization.py
'''
How to Train Your Dragon: V4
Sequentially initialize FT reactive distillation model automatically
'''
# system imports
import sys
import os
import datetime
sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath('../..'))

import numpy as np
from matplotlib import pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# import pickle
import dill
from copy import deepcopy

# pyomo imports
from pyomo import environ as pe
from global_sets.component import m

from stages.reactive_stage import reactive_stage_rule
from stages.condenser_stage import condenser_stage_rule
from stages.reboiler_stage import reboiler_stage_rule

from utility.display_utility import beautify, beautify_reactive, HiddenLogs, HiddenPrints, plot_distribution
from utility.model_utility import add_dual, update_dual, delete_dual, check_DOF, check_iteration
from utility.model_utility import which_MPCC, select_MPCC, augmented_objective, add_solver
from utility.time_utility import create_filename_time, log_now, log_end

2018-07-26 03:15:29 - Start Program


In [2]:
logname = create_filename_time()
log_text_dir = './log/text/opt_'+logname+'.dat'
log_figure_dir = './log/figure/opt_'+logname+'.pdf'

In [3]:
with open('./log/model/stage_20_pf.pickle','rb') as f:
    model = dill.load(f)

In [4]:
for j in model.reactive:
    model.reactive[j].MPCC_P_pf.rho = 1
model.reboiler.MPCC_P_pf.rho = 1

In [5]:
# for j in model.reactive:
#     model.reactive[j].MPCC_P_NCP.epi = 1e-4
# model.reboiler.MPCC_P_NCP.epi = 1e-4

In [6]:
opt = add_solver(pe, max_iter = 500, warm_start = True, output = True, scale = True)

In [7]:
for j in model.reactive:
    select_MPCC(model.reactive[j],'pf')
select_MPCC(model.reboiler,'pf')

> Selected MPCC: reactive[1].MPCC_P_pf
s_L:  1.964812775101591e-08
s_V:  0.0

> Selected MPCC: reactive[2].MPCC_P_pf
s_L:  2.282828746889007e-08
s_V:  0.0

> Selected MPCC: reactive[3].MPCC_P_pf
s_L:  2.541409097349865e-08
s_V:  0.0

> Selected MPCC: reactive[4].MPCC_P_pf
s_L:  2.8964348141935536e-08
s_V:  0.0

> Selected MPCC: reactive[5].MPCC_P_pf
s_L:  3.6801820770708585e-08
s_V:  0.0

> Selected MPCC: reactive[6].MPCC_P_pf
s_L:  5.067497122028571e-08
s_V:  0.0

> Selected MPCC: reactive[7].MPCC_P_pf
s_L:  8.886416478990899e-08
s_V:  0.0

> Selected MPCC: reactive[8].MPCC_P_pf
s_L:  2.636894172205893e-07
s_V:  0.0

> Selected MPCC: reactive[9].MPCC_P_pf
s_L:  2.915820537690447e-07
s_V:  0.0

> Selected MPCC: reactive[10].MPCC_P_pf
s_L:  2.557694373298898e-07
s_V:  0.0

> Selected MPCC: reactive[11].MPCC_P_pf
s_L:  2.931749515530684e-07
s_V:  0.0

> Selected MPCC: reactive[12].MPCC_P_pf
s_L:  2.215560288433305e-07
s_V:  0.0

> Selected MPCC: reactive[13].MPCC_P_pf
s_L:  1.97995836015

In [8]:
model.del_component(model.obj)
model.obj = augmented_objective(pe,model,expr = sum(model.reactive[j].T for j in model.TRAY_reactive), sense = pe.maximize)

------------------------------------------------------------------------------------------------------------
> Obj = maximize
> reactive[8].T + reactive[9].T + reactive[10].T + reactive[11].T + reactive[12].T + reactive[14].T + reactive[15].T + reactive[16].T + reactive[17].T + reactive[18].T - reactive[1].MPCC_P_pf.pf - reactive[2].MPCC_P_pf.pf - reactive[3].MPCC_P_pf.pf - reactive[4].MPCC_P_pf.pf - reactive[5].MPCC_P_pf.pf - reactive[6].MPCC_P_pf.pf - reactive[7].MPCC_P_pf.pf - reactive[8].MPCC_P_pf.pf - reactive[9].MPCC_P_pf.pf - reactive[10].MPCC_P_pf.pf - reactive[11].MPCC_P_pf.pf - reactive[12].MPCC_P_pf.pf - reactive[13].MPCC_P_pf.pf - reactive[14].MPCC_P_pf.pf - reactive[15].MPCC_P_pf.pf - reactive[16].MPCC_P_pf.pf - reactive[17].MPCC_P_pf.pf - reactive[18].MPCC_P_pf.pf - reactive[19].MPCC_P_pf.pf - reactive[20].MPCC_P_pf.pf - reboiler.MPCC_P_pf.pf
------------------------------------------------------------------------------------------------------------


In [9]:
results = opt.solve(model,tee=True)
update_dual(pe,model)

Ipopt 3.12.8: print_user_options=yes
linear_solver=ma86
linear_system_scaling=mc19
max_iter=500
warm_start_init_point=yes
warm_start_bound_push=1e-20
warm_start_mult_bound_push=1e-20
mu_init=1e-06
output_file=./tmp/ipopt_output_tmp.output
linear_scaling_on_demand=no


List of user-set options:

                                    Name   Value                used
                linear_scaling_on_demand = no                    yes
                           linear_solver = ma86                  yes
                   linear_system_scaling = mc19                  yes
                                max_iter = 500                   yes
                                 mu_init = 1e-06                 yes
                             output_file = ./tmp/ipopt_output_tmp.output  yes
                      print_user_options = yes                   yes
                   warm_start_bound_push = 1e-20                 yes
                   warm_start_init_point = yes                   yes
     

In [10]:
# pdf = PdfPages(log_figure_dir)

In [11]:
with HiddenLogs(log_text_dir,'w'):
    print('\n>','Original 20 stage case')
    print('-'*108)
    beautify(pe,model)
    log_now()

# plot_distribution(model,pdf,'Original 20 stage case')


> Original 20 stage case
------------------------------------------------------------------------------------------------------------
Here comes the result:
Total Conversion: 81.20%
------------------------------------------------------------------------------------------------------------
stages       T      Q                                            V               L       P            W     
condenser    30.00  -134.                                        3.0267          0.6407  0.0337       2.0654

stages       T      Q           r_FT   Conv%  F      cat         V       Re      L       P            P_VLE 
NON--[1]     119.3  0.000       0.000  0.000  0.000  0.000       5.7667  0.0000  0.7437  0.0000       20.000
NON--[2]     131.1  0.000       0.000  0.000  0.000  0.000       5.8697  0.0000  0.6752  0.0000       20.000
NON--[3]     140.3  0.000       0.000  0.000  0.000  0.000       5.8012  0.0000  0.6136  0.0000       20.000
NON--[4]     148.7  0.000       0.000  0.000  0.000  

# Optimization Input

**Parameters:**
* Stage Temperature
    * Rectifying section: 200C - 300C
    * Stripping section: 200C - 300C
* Reflux
    * Distillate / (Distillate + Reflux): 0.5 - 0.05, Refulx Ratio: 1 - 19
* Side-draw ratio
    * PR_L: 0 - 1

**Constraints:**
* Product
    * Distillate: C5~C7 >= 0.75
    * Gasoline: C8~C12 >= 0.75
    * Diesel: C13~C18 >= 0.6
    * Heavy: C19+ >= 0.85
    
**Objective:**
* Max gasoline production

### Parameters

In [12]:
for j in model.TRAY_reactive:
    model.reactive[j].T.setlb(200+273.15)
    model.reactive[j].T.setub(300+273.15)
    
model.condenser.PR_L.unfix()
model.condenser.PR_L.setlb(0.05)
model.condenser.PR_L.setub(0.5)

In [13]:
model.reactive[4].PR_L.unfix()
model.reactive[7].PR_L.unfix()
model.reactive[10].PR_L.unfix()

### Constraints

In [14]:
model.quality_coefficient = pe.Param(within=pe.NonNegativeReals,mutable=True,initialize=1)

In [15]:
model.product_spec_con = pe.ConstraintList()

In [16]:
model.product_spec_con.add(expr = sum(model.condenser.x[i] for i in m.PRODUCT_cnumber['naphtha']) >= \
                           model.quality_coefficient*0.75*sum(model.condenser.x[i] for i in m.COMP_ORG));
model.product_spec_con.add(expr = sum(model.reactive[7].x[i] for i in m.PRODUCT_cnumber['gasoline']) >= \
                           model.quality_coefficient*0.75*sum(model.reactive[7].x[i] for i in m.COMP_ORG));
model.product_spec_con.add(expr = sum(model.reactive[10].x[i] for i in m.PRODUCT_cnumber['diesel']) >= \
                           model.quality_coefficient*0.6*sum(model.reactive[10].x[i] for i in m.COMP_ORG));
model.product_spec_con.add(expr = sum(model.reboiler.x[i] for i in m.PRODUCT_cnumber['heavy']) >= \
                           model.quality_coefficient*0.85*sum(model.reboiler.x[i] for i in m.COMP_ORG));

### Objective

In [17]:
model.del_component(model.obj)
model.obj = augmented_objective(pe,model,expr = model.reactive[7].L['P'], sense = pe.maximize)

------------------------------------------------------------------------------------------------------------
> Obj = maximize
> reactive[7].L[P] - reactive[1].MPCC_P_pf.pf - reactive[2].MPCC_P_pf.pf - reactive[3].MPCC_P_pf.pf - reactive[4].MPCC_P_pf.pf - reactive[5].MPCC_P_pf.pf - reactive[6].MPCC_P_pf.pf - reactive[7].MPCC_P_pf.pf - reactive[8].MPCC_P_pf.pf - reactive[9].MPCC_P_pf.pf - reactive[10].MPCC_P_pf.pf - reactive[11].MPCC_P_pf.pf - reactive[12].MPCC_P_pf.pf - reactive[13].MPCC_P_pf.pf - reactive[14].MPCC_P_pf.pf - reactive[15].MPCC_P_pf.pf - reactive[16].MPCC_P_pf.pf - reactive[17].MPCC_P_pf.pf - reactive[18].MPCC_P_pf.pf - reactive[19].MPCC_P_pf.pf - reactive[20].MPCC_P_pf.pf - reboiler.MPCC_P_pf.pf
------------------------------------------------------------------------------------------------------------


In [18]:
itr_count1 = 0
meta_results = {}
while True:
    itr_count1 += 1
    opt = add_solver(pe, max_iter = itr_count1, warm_start = True, output = True, scale = True)
    model_tmp = deepcopy(model)
    results = opt.solve(model_tmp,tee=False)
    model_tmp.solutions.store_to(results)
    meta_results[itr_count1] = results
    print('itr_count',itr_count1,'obj',model_tmp.obj())
    if results.solver.termination_condition.key == 'optimal':
        break

	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 1 obj 0.03900315600810916
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 2 obj 0.03900491278074784
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 3 obj 0.03879732686662436
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 4 obj 0.03892804917247923
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 5 obj 0.03924548544941896
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 6 obj 0.04078339109520846
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 7 obj 0.041109258370368766
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 8 obj 0.04844322703976604
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 9 obj 0.

itr_count 40 obj 0.10423843889590552
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 41 obj 0.10374171647577955
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 42 obj 0.10393970381466405
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 43 obj 0.10502717818085844
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 44 obj 0.10539231156179263
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 45 obj 0.10482516552989656
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 46 obj 0.1048010552977964
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 47 obj 0.10473115047939138
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 48 obj 0.10453123520907744
	message from solver=Ipopt 3.12.8\x3a Maximum Number

In [19]:
results = opt.solve(model,tee=True)
update_dual(pe,model)

Ipopt 3.12.8: print_user_options=yes
linear_solver=ma86
linear_system_scaling=mc19
max_iter=61
warm_start_init_point=yes
warm_start_bound_push=1e-20
warm_start_mult_bound_push=1e-20
mu_init=1e-06
output_file=./tmp/ipopt_output_tmp.output
linear_scaling_on_demand=no


List of user-set options:

                                    Name   Value                used
                linear_scaling_on_demand = no                    yes
                           linear_solver = ma86                  yes
                   linear_system_scaling = mc19                  yes
                                max_iter = 61                    yes
                                 mu_init = 1e-06                 yes
                             output_file = ./tmp/ipopt_output_tmp.output  yes
                      print_user_options = yes                   yes
                   warm_start_bound_push = 1e-20                 yes
                   warm_start_init_point = yes                   yes
      

	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.


In [20]:
with HiddenLogs(log_text_dir):
    print('\n>','Optimized Product Side Draw')
    print('-'*108)
    beautify(pe,model)
    # check_product_spec(model)
    log_now()

# plot_distribution(model,pdf,'Optimized Product Side Draw')


> Optimized Product Side Draw
------------------------------------------------------------------------------------------------------------
Here comes the result:
Total Conversion: 87.56%
------------------------------------------------------------------------------------------------------------
stages       T      Q                                            V               L       P            W     
condenser    30.00  -157.                                        2.1436          0.8942  0.0865       2.3403

stages       T      Q           r_FT   Conv%  F      cat         V       Re      L       P            P_VLE 
NON--[1]     129.1  0.000       0.000  0.000  0.000  0.000       5.4649  0.0000  1.0901  0.0000       20.000
NON--[2]     142.6  0.000       0.000  0.000  0.000  0.000       5.6607  0.0000  1.0321  0.0000       20.000
NON--[3]     152.2  0.000       0.000  0.000  0.000  0.000       5.6027  0.0000  0.9880  0.0000       20.000
NON--[4]     160.3  0.000       0.000  0.000  0.

# Open up feed allocation

**Parameters:**
* Stage Temperature
    * Rectifying section: 200C - 300C
    * Stripping section: 200C - 300C
* Reflux
    * Distillate / (Distillate + Reflux): 0.5 - 0.05, Refulx Ratio: 1 - 19
* Side-draw
    * PR_L: 0 - 1
    
* Feed
    * F: 0 - 3

**Constraints:**
* Product
    * Distillate: C5~C7 >= 0.75
    * Gasoline: C8~C12 >= 0.75
    * Diesel: C13~C18 >= 0.6
    * Heavy: C19+ >= 0.85
    
* Total feed
    * Total feed = 10 kmol/s
    
**Objective:**
* Max gasoline production

In [21]:
for j in model.reactive:
    model.reactive[j].F.unfix()
    model.reactive[j].F.setlb(0)
    model.reactive[j].F.setub(10)

In [22]:
model.total_feed_con = pe.ConstraintList()

In [23]:
model.total_feed_con.add(expr = sum(model.reactive[j].F for j in model.reactive) == 10);

In [24]:
itr_count2 = 0
while True:
    itr_count2 += 1
    opt = add_solver(pe, max_iter = itr_count2, warm_start = True, output = True, scale = True)
    model_tmp = deepcopy(model)
    results = opt.solve(model_tmp,tee=False)
    model_tmp.solutions.store_to(results)
    meta_results[itr_count2+itr_count1] = results
    print('itr_count',itr_count2,'obj',model_tmp.obj())
    if results.solver.termination_condition.key == 'optimal':
        break

	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 1 obj 0.1023969006650274
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 2 obj 0.1024177579047409
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 3 obj 0.10351003223134445
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 4 obj 0.10516117089242027
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 5 obj 0.10561669369039436
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 6 obj 0.1072820647973047
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 7 obj 0.1081193221247027
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 8 obj 0.10817809084075378
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 9 obj 0.10900

itr_count 40 obj 0.12228179825572207
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 41 obj 0.1223050120398111
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 42 obj 0.1223838765725557
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 43 obj 0.12242629858342262
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 44 obj 0.1224676593360154
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 45 obj 0.12250870207555739
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 46 obj 0.12255724638969295
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 47 obj 0.12261211710443753
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 48 obj 0.12253286018922037
	message from solver=Ipopt 3.12.8\x3a Maximum Number o

In [25]:
with open('./meta_results_pf_2step_F16.pickle','wb') as f:
    dill.dump(meta_results,f)

In [26]:
results = opt.solve(model,tee=True)
update_dual(pe,model)

Ipopt 3.12.8: print_user_options=yes
linear_solver=ma86
linear_system_scaling=mc19
max_iter=78
warm_start_init_point=yes
warm_start_bound_push=1e-20
warm_start_mult_bound_push=1e-20
mu_init=1e-06
output_file=./tmp/ipopt_output_tmp.output
linear_scaling_on_demand=no


List of user-set options:

                                    Name   Value                used
                linear_scaling_on_demand = no                    yes
                           linear_solver = ma86                  yes
                   linear_system_scaling = mc19                  yes
                                max_iter = 78                    yes
                                 mu_init = 1e-06                 yes
                             output_file = ./tmp/ipopt_output_tmp.output  yes
                      print_user_options = yes                   yes
                   warm_start_bound_push = 1e-20                 yes
                   warm_start_init_point = yes                   yes
      

  66 -1.2351744e-01 3.69e+01 1.15e+01  -9.0 3.81e+03    -  8.24e-02 4.04e-01h  1
  67 -1.2354383e-01 3.89e+01 1.04e+01  -9.0 5.91e+03    -  3.99e-02 1.13e-01h  1
  68 -1.2359005e-01 5.39e+01 1.46e+05  -9.0 6.26e+03    -  5.76e-06 2.04e-01h  1
  69 -1.2360747e-01 5.37e+01 1.32e+05  -9.0 6.18e+03    -  3.82e-03 9.10e-02h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  70 -1.2362194e-01 4.37e+01 9.69e+04  -9.0 1.97e+03    -  1.15e-03 2.69e-01h  1
  71 -1.2362278e-01 4.36e+01 9.66e+04  -9.0 1.06e+04    -  1.42e-01 2.74e-03h  1
  72 -1.2364841e-01 3.77e+01 4.79e+04  -9.0 1.99e+03    -  4.72e-01 5.04e-01h  1
  73 -1.2365156e-01 3.55e+01 4.48e+04  -9.0 1.88e+03    -  2.27e-02 6.60e-02h  1
  74 -1.2364608e-01 1.64e+01 2.05e+04  -9.0 1.82e+02    -  5.71e-02 5.41e-01h  1
  75 -1.2364303e-01 7.13e+00 8.90e+03  -9.0 9.42e+01    -  5.40e-01 5.67e-01h  1
  76 -1.2364041e-01 2.27e-02 1.13e-01  -9.0 4.46e+01    -  9.97e-01 1.00e+00h  1
  77 -1.2364031e-01 1.23e-05

In [27]:
with HiddenLogs(log_text_dir):
    print('\n>','Optimized Feed Amount and Location')
    print('-'*108)
    beautify(pe,model)
    # check_product_spec(model)
    log_now()

# plot_distribution(model,pdf,'Optimized Feed Amount and Location')


> Optimized Feed Amount and Location
------------------------------------------------------------------------------------------------------------
Here comes the result:
Total Conversion: 89.06%
------------------------------------------------------------------------------------------------------------
stages       T      Q                                            V               L       P            W     
condenser    30.00  -147.                                        1.9106          0.6176  0.0990       2.3902

stages       T      Q           r_FT   Conv%  F      cat         V       Re      L       P            P_VLE 
NON--[1]     124.2  0.000       0.000  0.000  0.000  0.000       5.0176  0.0000  0.7252  0.0000       20.000
NON--[2]     133.6  0.000       0.000  0.000  0.000  0.000       5.1252  0.0000  0.6728  0.0000       20.000
NON--[3]     140.9  0.000       0.000  0.000  0.000  0.000       5.0728  0.0000  0.6341  0.0000       20.000
NON--[4]     147.5  0.000       0.000  0.

# Increase spec a little bit

**Parameters:**
* Stage Temperature
    * Rectifying section: 200C - 300C
    * Stripping section: 200C - 300C
* Reflux
    * Distillate / (Distillate + Reflux): 0.5 - 0.05, Refulx Ratio: 1 - 19
* Side-draw
    * PR_L: 0 - 1
    
* Feed
    * F: 0 - 3

**Constraints:**
* Product
    * Distillate: C5~C7 >= 0.75
    * Gasoline: C8~C12 >= **0.8**
    * Diesel: C13~C18 >= 0.6
    * Heavy: C19+ >= 0.85
    
* Total feed
    * Total feed = 10 kmol/s
    
**Objective:**
* Max gasoline production

In [28]:
# model.product_spec_con.add(expr = sum(model.reactive[7].x[i] for i in product['gasoline']) >= \
#                            model.quality_coefficient*0.8*sum(model.reactive[7].x[i] for i in m.COMP_ORG));

In [29]:
# results = opt.solve(model,tee=True)
# update_dual(pe,model)

In [30]:
# with HiddenLogs(log_text_dir):
#     print('\n>','Increase Gasoline Specification to 80%')
#     print('-'*108)
#     beautify(pe,model)
#     check_product_spec(model)
#     log_now()

# plot_distribution(model,pdf,'Increase Gasoline Specification to 80%')

# Open up catalyst allocation ( = TOTAL reconstruction, long wait time)

**Parameters:**
* Stage Temperature
    * Rectifying section: 200C - 300C
    * Stripping section: 200C - 300C
* Reflux
    * Distillate / (Distillate + Reflux): 0.5 - 0.05, Refulx Ratio: 1 - 19
* Side-draw
    * PR_L: 0 - 1
    
* Feed
    * F: 0 - 3
        
* Catalyst
    * cat: 0 - 10000

**Constraints:**
* Product
    * Distillate: C5~C7 >= 0.75
    * Gasoline: C8~C12 >= 0.75 / 0.8
    * Diesel: C13~C18 >= 0.6
    * Heavy: C19+ >= 0.85
    
* Total feed
    * sum feed = 10 kmol/s
    
* Total catalyst
    * sum cat = 30000 kg
    
**Objective:**
* Max gasoline production

In [31]:
# for j in model.reactive:
#     model.reactive[j].cat.unfix()
#     model.reactive[j].cat.setlb(0)
#     model.reactive[j].cat.setub(30000)

In [32]:
# model.total_cat_con = pe.ConstraintList()

In [33]:
# model.total_cat_con.add(expr = sum(model.reactive[j].cat for j in model.reactive) == 10*3000);

In [34]:
# results = opt.solve(model,tee=True)
# update_dual(pe,model)

In [35]:
# with HiddenLogs(log_text_dir):
#     print('\n>','Optimized Catalyst Amount and Location')
#     print('-'*108)
#     beautify(pe,model)
#     check_product_spec(model)
#     log_now()

# plot_distribution(model,pdf,'Optimized Catalyst Amount and Location')

In [36]:
# pdf.close()

In [37]:
# with open('./log/model/{}.pickle'.format(logname),'wb') as f:
#     dill.dump(model,f)

In [38]:
# for j in model.TRAY_reactive:
#     print('stage',j,'\n')
#     print('Total Flow\t{}'.format(model.reactive[j].F.value + sum(model.reactive[j].r_total_comp[i].value for i in m.COMP_TOTAL)))
#     for i in m.COMP_TOTAL:
#         if i in m.COMP_FEED:
#             print('{}\t\t{}'.format(i,model.reactive[j].F.value*model.reactive[j].z[i].value + model.reactive[j].r_total_comp[i].value))
#         else:
#             print('{}\t\t{}'.format(i,model.reactive[j].r_total_comp[i].value))